In [1]:
import pandas as pd

#### Data Background

In [2]:
# There are seven files containing data approporiate for initial review,
# descriptions of which can be found in the comment sections below.
#
# Tables are each provided from http://efigi.org 
#
# Table history is as follows:
# Mar 29, 2011: initial 1.6 release.
# May 16, 2011: v1.6.1 fixes coordinate swaps in EFIGI_coord_redshift table.
# Jul 28, 2011: v1.6.2 fixes corrupted B/T values for some early types.
#
# This archive contains version 1.6.2 of the EFIGI reference dataset. The EFIGI
# reference dataset contains SDSS images and visual morphology for 4458
# galaxies from the RC3 catalogue. See Baillard et al. 2011 (A&A 532, A74) and
# http://efigi.org for details.
#
# Any use of these data must reference :
# Baillard, B., Bertin, E., de Lapparent, V., et al. 2011, A&A, 532, A74
#
# Comments on table content should be sent to the EFIGI user forum: 
# http://efigi.org/forum
# or to Valerie de Lapparent (lapparen at iap.fr) and Emmanuel Bertin (bertin at iap.fr)
#
# The EFIGI dataset is distributed in 7 separate compressed archives (gzipped
# tar format):
# - efigi_tables-1.6.2.tgz: 6 ASCII tables, including morphological information
# - efigi_ima_u-1.6.tgz: 4458 galaxy images in the SDSS u-band (FITS format)
# - efigi_ima_g-1.6.tgz: 4458 galaxy images in the SDSS g-band (FITS format)
# - efigi_ima_r-1.6.tgz: 4458 galaxy images in the SDSS r-band (FITS format)
# - efigi_ima_i-1.6.tgz: 4458 galaxy images in the SDSS i-band (FITS format)
# - efigi_ima_z-1.6.tgz: 4458 galaxy images in the SDSS z-band (FITS format)
# - efigi_psf-1.6.tgz: 4458x5 PSF images

In [3]:
# EFIGI_attributes.txt version 1.6.2 :
#
# This file contains the PGC name, the EFIGI attributes and 
# the corresponding lower and upper bounds of confidence intervals
# for the 4458 galaxies of the EFIGI catalogue
#
# Description  
#--------------
#
#    PGC_name              PGC name		
#    T                     EFIGI morphological type 
#    T_inf         	   Lower confidence limit in T
#    T_sup                 Upper confidence limit in T
#    Bulge_to_Total        Bulge-to-total ratio
#    Bulge_to_Total_inf    Lower confidence limit in Bulge_to_Total
#    Bulge_to_Total_sup    Upper confidence limit in Bulge_to_Total
#    Arm_Strength          Strength of spiral arms
#    Arm_Strength_inf      Lower confidence limit in Arm_Strength
#    Arm_Strength_sup      Upper confidence limit in Arm_Strength
#    Arm_Curvature         Average curvature of the spiral arms 
#    Arm_Curvature_inf     Lower confidence limit in Arm_Curvature
#    Arm_Curvature_sup     Upper confidence limit in Arm_Curvature
#    Arm_Rotation          Direction of winding of the spiral arms 
#    Arm_Rotation_inf      Lower confidence limit in Arm_Rotation
#    Arm_Rotation_sup      Upper confidence limit in Arm_Rotation
#    Bar_Length            Length of central bar   
#    Bar_Length_inf        Lower confidence limit in Bar_Length
#    Bar_Length_sup        Upper confidence limit in Bar_Length
#    Inner_Ring            Strength of inner ring, lens or inner pseudo-ring
#    Inner_Ring_inf        Lower confidence limit in Inner_Ring  
#    Inner_Ring_sup        Upper confidence limit in Inner_Ring 
#    Outer_Ring            Strength of outer ring 
#    Outer_Ring_inf        Lower confidence limit in Outer_Ring
#    Outer_Ring_sup        Upper confidence limit in Outer_Ring
#    Pseudo_Ring           Type and strength of outer pseudo-ring 
#    Pseudo_Ring_inf       Lower confidence limit in Pseudo_Ring
#    Pseudo_Ring_sup       Upper confidence limit in Pseudo_Ring
#    Perturbation          Deviation from rotational symmetry
#    Perturbation_inf      Lower confidence limit in Perturbation
#    Perturbation_sup      Upper confidence limit in Perturbation
#    Visible_Dust          Strength of dust features 
#    Visible_Dust_inf      Lower confidence limit in Visible_Dust
#    Visible_Dust_sup      Upper confidence limit in Visible_Dust
#    Dust_Dispersion       Patchiness of dust features
#    Dust_Dispersion_inf   Lower confidence limit in Dust_Dispersion
#    Dust_Dispersion_sup   Upper confidence limit in Dust_Dispersion
#    Flocculence           Strength of scattered HII regions  
#    Flocculence_inf       Lower confidence limit in Flocculence
#    Flocculence_sup       Upper confidence limit in Flocculence
#    Hot_Spots             Strength of regions of strong star formation, active nuclei, or stellar nuclei 
#    Hot_Spots_inf         Lower confidence limit in Hot_Spots
#    Hot_Spots_sup         Upper confidence limit in Hot_Spots
#    Inclination           Inclination of disks or elongation of spheroids 
#    Inclination_inf       Lower confidence limit in Inclination
#    Inclination_sup       Upper confidence limit in Inclination
#    Contamination         Severity of contamination by stars, galaxies or artifacts
#    Contamination_inf     Lower confidence limit in Contamination
#    Contamination_sup     Upper confidence limit in Contamination
#    Multiplicity          Abundance of neighbouring galaxies 
#    Multiplicity_inf      Lower confidence limit in Multiplicity
#    Multiplicity_sup      Upper confidence limit in Multiplicity
#
# Notes  
#-------
#
#    Objects are ordered with increasing PGC number
#    Attribute values and confidence limits take values : 0, 0.25, 0.5, 0.75, or 1.0
#    Attribute and confidence limits of -1 are undetermined

In [4]:
# EFIGI_coord_redshift.txt version 1.6.2 :
#
# This file contains the PGC name, the J2000 EFIGI corrected coordinates, the EFIGI 
# selected redshifts and corresponding distances for the 4458 galaxies of 
# the EFIGI catalogue
#
# Description  
#--------------
#
#    PGC_name        PGC name
#    RA       	     Right ascension J2000 (degrees)
#    DEC   	         Declination J2000 (degrees)
#    z_hel           Selected heliocentric redshift 
#    z_hel_err       Uncertainty in selected heliocentric redshift 
#    z_hel_cat       Catalogue of redshift correction for Local Group infall into Virgo redshift 
#    z_dis           Selected redshift corrected for Local Group infall into Virgo 
#    z_dis_err       Uncertainty in selected redshift corrected for Local Group infall into Virgo 
#    z_dis_cat       Catalogue of redshift correction for Local Group infall into Virgo redshift 
#    z_all           z_dis, or z_hel if z_dis undertermined 
#    z_all_type      Type of redshift used for z_all 
#    z_all_err       Uncertainty in redshift used for z_all 
#    z_all_err_type  Type of redshift used for uncertainty in z_all 
#    D_com           Comoving distance derived from z_all (Mpc) 
#    D_lum           Luminosity distance derived from z_all (Mpc) 
#    D_diam          Transverse diameter distance derived z_all (Mpc) 
#
# Notes  
#-------
#
#  z_hel_cat =      HL if HyperLeda heliocentric redshift stored in z_hel
#            =      NED if NED      heliocentric redshift stored in z_hel
#            =      SDSS if SDSS    heliocentric redshift stored in z_hel
#  z_dis_cat =      HL if stored in z_dis the HyperLeda corrected redshift for Local Group infall into Virgo
#            =      NED-vir-HL if stored in z_dis the NED heliocentric redshift corrected with the HyperLeda correction 
#                   for Local Group infall into Virgo
#  z_all_type =     DIS if z_dis stored in z_all
#             =     HEL if z_hel stored in z_all
#  z_all_err_type = HEL if z_hel_err is stored in z_all_err
#                   DIS if z_dis_err is stored in z_all_err

In [5]:
# EFIGI_HyperLeda.txt version 1.6.2 :
#
# This file contains the PGC name, the HyperLeda velocities, redshifts, 
# types and names for the 4458 galaxies of the EFIGI catalogue
#
# Description  
#--------------
#
#      PGC_name	     PGC name		
#      PGC_no	     PGC number		
#      vrad	         Heliocentric radial velocity from radio measurement (km/s)
#      e_vrad	     Actual error on vrad (km/s)
#      vopt	         Heliocentric radial velocity from optical measurement (km/s)
#      e_vopt	     Actual error on vopt (km/s)
#      v	         Mean heliocentric radial velocity (km/s)
#      e_v	         Actual error on v (km/s)
#      vvir	         Radial velocity corrected for Local Group infall into Virgo (km/s)
#      zvir	         Redshift corrected for Local Group infall into Virgo 
#      z_err         Redshit error derived from e_v 
#      type          Morphological type 
#      objname	     Principal designation
#      hl_names      List of all object names
#
# Notes  
#-------
#
#    Objects are ordered with increasing PGC number
#    Undefined values of velocity are listed as -9999
#    Undefined values of redshift are listed as -99.99

In [6]:
# EFIGI_NED.txt version 1.6.2:
#
# This file contains the PGC name, the NED velocity, redshift and object name
# for the 4458 galaxies of the EFIGI catalogue
#
# Description  
#--------------
#
#    PGC_name      PGC name		
#    cz	           Heliocentric velocity (km/s) 
#    redshift 	   Redshift
#    nedname       Object name 		
#
# Notes  
#-------
#
#    Objects are ordered with increasing PGC number
#    Undefined values of velocity are listed as -9999
#    Undefined values of redshift are listed as -99.99

In [7]:
# EFIGI_PGC.txt version 1.6.2 :
#
# This file contains the PGC name and other PGC parameters for the 
# 4458 galaxies of the EFIGI catalogue
#
# Description  
#--------------
#
#   PGC_name      PGC name
#   T_PGC         RC3 morphological type
#   e_T_PGC       Mean error on T_PGC
#   type	      Expanded morphological type			
#   D25           log(D_{25}), decimal logarithm of mean apparent major
#                 isophotal diameter measured at or reduced to surface brightness
#                 level 25.0 B/mag^2 (in units of 0.1 arcmin)
#   R25           log(R_{25}) decimal logarithm of ratio of mean major isophotal 
#                 diameter, D_{25}, to mean minor isophotal diameter measured at 
#                 or reduced to the surface brightness level 25.0 B/mag^2
#   PA		      Position angle of major axis (degrees)
#   B_T_mag	      Total B magnitude
#   e_B_T_mag     Mean error on B_T_mag
#   B_V_T	      Total (B-V)
#   e_B_V_T	      Mean error on total (B-V)
#   cz		      Heliocentric velocity (km/s)
#   z             Redshift  
#
# Notes  
#-------
#
#    Undefined values of D25, R25, B_T_mag, e_B_T_mag, B_V_T, e_B_V_T, z, are listed as -99.99
#    Undefined values of PA and cz are listed as -9999

In [8]:
# EFIGI_SDSS.txt version 1.6.2 :
#
# This file contains the PGC name, the unique SDSS photometric identifier 
# and SDSS spectroscopic identifier, the SDSS redshift, uncertainty and 
# confidence level, for the 4458 galaxies of the EFIGI catalogue
#
# Description  
#--------------
#
#      PGC_name	     PGC name		
#      objID	     Unique SDSS photometric identifier composed from skyVersion, rerun, run, camcol, field, obj
#      specObjID     Unique SDSS spectroscopic identifier
#      z  	         Redshift 
#      zErr  	     Uncertainty in redshift 
#      zConf         Confidence level in redshift 
#
# Notes  
#-------
#
#    Objects are ordered with increasing PGC number
#    Undefined values of SDSS photometric and spectroscopic identifiers are listed as "none"
#    Undefined values of z, zErr and zConf are listed as -99.99

#### Import Data Txt Files to Dataframe

In [21]:
attributes = pd.read_csv('efigi-1.6/EFIGI_attributes.txt', sep=None)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


ParserError: Expected 163 fields in line 3, saw 199

In [10]:
coord_redshift = pd.read_csv('efigi-1.6/EFIGI_coord_redshift.txt')

In [11]:
hyperleda = pd.read_csv('efigi-1.6/EFIGI_HyperLeda.txt')

In [12]:
ned = pd.read_csv('efigi-1.6/EFIGI_NED.txt')

In [13]:
pgc = pd.read_csv('efigi-1.6/EFIGI_PGC.txt')

In [14]:
sdss = pd.read_csv('efigi-1.6/EFIGI_SDSS.txt')

In [15]:
attributes

,PGCname T T_inf T_sup Bulge_to_Total Bulge_to_Total_inf Bulge_to_Total_sup Arm_Strength Arm_Strength_inf Arm_Strength_sup Arm_Curvature Arm_Curvature_inf Arm_Curvature_sup Arm_Rotation Arm_Rotation_inf Arm_Rotation_sup Bar_Length Bar_Length_inf Bar_Length_sup Inner_Ring Inner_Ring_inf Inner_Ring_sup Outer_Ring Outer_Ring_inf Outer_Ring_sup Pseudo_Ring Pseudo_Ring_inf Pseudo_Ring_sup Perturbation Perturbation_inf Perturbation_sup Visible_Dust Visible_Dust_inf Visible_Dust_sup Dust_Dispersion Dust_Dispersion_inf Dust_Dispersion_sup Flocculence Flocculence_inf Flocculence_sup Hot_Spots Hot_Spots_inf Hot_Spots_sup Inclination Inclination_inf Inclination_sup Contamination Contamination_inf Contamination_sup Multiplicity Multiplicity_inf Multiplicity_sup
0,PGC0000212 1 0 2 0.5 0.5 0.5 0.25...
1,PGC0000218 0 -1 1 0.75 0.5 0.75 -1...
2,PGC0000243 -2 -3 -2 1 0.75 1 0...
3,PGC0000255 9 9 10 0 0 0.25 0...
4,PGC0000281 7 6 9 0.25 0 0.5 0...
...,...
4453,PGC0072806 6 4 8 0.25 0.25 0.25 0...
4454,PGC0072922 4 3 4 0.25 0.25 0.5 0.5...
4455,PGC0072927 3 2 4 0.5 0.5 0.75 0.25...
4456,PGC0072930 3 2 4 0.75 0.5 0.75 0...


In [16]:
coord_redshift

,PGC_name RA DEC z_hel z_hel_err z_hel_cat z_dis z_dis_err z_dis_cat z_all z_all_type z_all_err z_all_err_type D_com D_lum D_diam
0,PGC0000212 0.7970833 15.9650556 0.03746...
1,PGC0000218 0.8121250 16.1454167 0.003501...
2,PGC0000243 0.8839167 -10.7445833 0.02931...
3,PGC0000255 0.9299583 15.2183056 0.002929...
4,PGC0000281 1.0025000 -11.1777778 0.03833...
...,...
4453,PGC0072806 358.5445833 0.3938889 0.04987...
4454,PGC0072922 359.0163333 -0.9884722 0.02225...
4455,PGC0072927 359.0291667 -0.9169444 0.02428...
4456,PGC0072930 359.0325000 0.5491667 0.02224...


In [17]:
hyperleda

,PGC_name PGC_no vrad e_vrad vopt e_vopt v e_v vvir zvir z_err type objname hl_names
0,PGC0000212 212 11230.4 4.5...
1,PGC0000218 218 1050.3 4.8...
2,PGC0000243 243 -9999 -9999...
3,PGC0000255 255 878.1 4.1...
4,PGC0000281 281 -9999 -9999...
...,...
4453,PGC0072806 7.281e+04 -9999 -9999...
4454,PGC0072922 7.292e+04 6664.7 7.3...
4455,PGC0072927 7.293e+04 7279.8 7.7...
4456,PGC0072930 7.293e+04 6668 5.2...


In [18]:
ned

,PGC_name cz redshift nedname
0,PGC000212 11231 0.03746 IC_5381
1,PGC000218 1050 0.003502 NGC_7814
2,PGC000243 8787 0.02931 NGC_7808
3,PGC000255 878 0.002929 UGC_00017
4,PGC000281 11491 0.03833 MCG_-02-01-015
...,...
4453,PGC072806 14950 0.04987 UGC_12837/NO...
4454,PGC072922 6665 0.02223 IC_1515
4455,PGC072927 7286 0.0243 IC_1516
4456,PGC072930 6664 0.02223 NGC_7787


In [19]:
pgc

,PGC_name T_PGC T_PGC_err type D25 R25 PA B_T_mag e_B_T_mag B_V_T e_B_V_T cz z
0,PGC0000212 2 1.8 .S..2$/...
1,PGC0000218 2 0.3 .SAS2*/...
2,PGC0000243 -2 1.2 PLA.0*....
3,PGC0000255 9.3 0.6 .S..9*....
4,PGC0000281 1 1.7 .S..1?....
...,...
4453,PGC0072806 -1 0.9 .L..+$....
4454,PGC0072922 2 0.5 PSBT2.....
4455,PGC0072927 4.3 0.5 .S..4P....
4456,PGC0072930 0 0.6 PSBT0*....


In [20]:
sdss

,PGC_name objId SpecObjId z zErr zConf
0,PGC0000212 587730775499735086 2113305826872524...
1,PGC0000218 587727223561388198 none ...
2,PGC0000243 587727178449879045 1831826873877790...
3,PGC0000255 587727222487646439 2113305828088872...
4,PGC0000281 587727177913008262 1831826874255278...
...,...
4453,PGC0072806 587731186740166867 1927544707722772...
4454,PGC0072922 587731185129750564 none ...
4455,PGC0072927 587731185129750586 1091532578629877...
4456,PGC0072930 588015509805793362 1088717714106613...
